In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")

In [12]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats

from vaep.domain.vaep.vaep import *

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load VAEP Data

In [13]:
vaep_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/schema_chains_vaep_values.csv")
vaep_data.head()

,match_id,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,scores,concedes,offensive_value,defensive_value,vaep_value,Season
0,202101_BrisbaneLions_Sydney,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.052962,0.009920,0.000000,-0.000000,0.000000,2021
1,202101_BrisbaneLions_Sydney,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.022556,0.050321,-0.030406,-0.040401,-0.070807,2021
2,202101_BrisbaneLions_Sydney,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.074422,0.008536,0.024101,0.014020,0.038121,2021
3,202101_BrisbaneLions_Sydney,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.084299,0.013648,0.009877,-0.005111,0.004766,2021
4,202101_BrisbaneLions_Sydney,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.215944,0.012438,0.131645,0.001209,0.132854,2021


In [34]:
vaep_data['Season'] = vaep_data['match_id'].apply(lambda x: int(x[:4]))
vaep_2021 = vaep_data[vaep_data['Season'] == 2021]
vaep_2022 = vaep_data[vaep_data['Season'] == 2022]
vaep_2023 = vaep_data[vaep_data['Season'] == 2023]

In [35]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/player_stats_202319.csv")

In [36]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
player_stats_2022 = player_stats[player_stats['Season'] == 2022]
player_stats_2023 = player_stats[player_stats['Season'] == 2023]

Individual Actions

In [15]:
vaep_data['vaep_value'].max(), vaep_data['vaep_value'].min()

(1.0781667708285323, -0.9445851312802004)

Match Level

In [28]:
def get_match_player_level_vaep(vaep_data, match_id, shots = True):
    
    match = get_match(vaep_data, match_id)
    if shots == False:
        match = match[match['action_type'] != "Shot"]
    match_vaep = get_player_vaep(match)
    match_vaep_ranking = get_match_rankings(match_vaep)
    
    return match_vaep_ranking

In [29]:
match_id = "2022F4_Geelong_Sydney"

In [32]:
match_vaep_ranking = get_match_player_level_vaep(vaep_data, match_id)
match_vaep_ranking.sort_values('vaep_value', ascending = False).head()

,player,team,vaep_value,offensive_value,defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_vaep_value,normalised_offensive_value,normalised_defensive_value,z_vaep_value,z_offensive_value,z_defensive_value
13,Isaac Smith,Geelong,3.365278,3.429636,-0.064358,69,1.0,1.0,35.0,1.000000,1.000000,0.234744,3.048515,3.010883,-0.644855
42,Tyson Stengle,Geelong,3.056119,3.253465,-0.197345,34,2.0,2.0,44.0,0.926418,0.958225,0.095051,2.682661,2.815915,-1.238274
5,Chad Warner,Sydney,3.033626,3.222696,-0.189071,69,3.0,3.0,43.0,0.921065,0.950929,0.103743,2.656042,2.781864,-1.201349
0,Brad Close,Geelong,2.101191,1.890228,0.210962,44,4.0,5.0,9.0,0.699140,0.634965,0.523945,1.552612,1.307231,0.583679
24,Mark Blicavs,Geelong,1.848786,1.919636,-0.070850,49,5.0,4.0,36.0,0.639066,0.641939,0.227924,1.253920,1.339776,-0.673827


In [33]:
match_noshot_vaep_ranking = get_match_player_level_vaep(vaep_data, match_id, shots=False)
match_noshot_vaep_ranking.sort_values('vaep_value', ascending = False).head()

,player,team,vaep_value,offensive_value,defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_vaep_value,normalised_offensive_value,normalised_defensive_value,z_vaep_value,z_offensive_value,z_defensive_value
24,Mark Blicavs,Geelong,1.911770,1.947344,-0.035574,47,1.0,1.0,34.0,1.000000,1.000000,0.278144,2.805507,2.762567,-0.533768
5,Chad Warner,Sydney,1.797161,1.947089,-0.149928,66,2.0,2.0,40.0,0.952275,0.999893,0.160177,2.570736,2.762094,-1.044359
13,Isaac Smith,Geelong,1.564872,1.549475,0.015398,65,3.0,4.0,27.0,0.855545,0.833371,0.330727,2.094903,2.024451,-0.306176
30,Patrick Dangerfield,Geelong,1.451149,1.756346,-0.305197,57,4.0,3.0,46.0,0.808188,0.920010,0.000000,1.861945,2.408233,-1.737643
42,Tyson Stengle,Geelong,1.138203,1.298363,-0.160161,29,5.0,5.0,42.0,0.677871,0.728205,0.149620,1.220889,1.558597,-1.090052


In [38]:
match_stats = player_stats[player_stats['Match_ID'] == match_id]
match_stats[['Team', 'Player', "Goals", "Behinds", "Disposals"]]

,Team,Player,Goals,Behinds,Disposals
18967,Geelong,Jack Henry,0,0,10
18968,Sydney,Hayden McLean,1,0,4
18969,Sydney,Dylan Stephens,0,0,12
18970,Geelong,Zach Guthrie,0,1,15
18971,Geelong,Gryan Miers,0,1,18
18972,Sydney,Nick Blakey,0,0,15
18973,Sydney,James Rowbottom,0,0,16
18974,Geelong,Mark O'Connor,0,0,13
18975,Sydney,Tom McCartin,0,0,9
18976,Geelong,Sam De Koning,1,0,16


Player Ratings

In [39]:
players_vaep = get_player_vaep(vaep_data)
players_vaep.sort_values('vaep_value', ascending = False).head(5)

,player,team,vaep_value,offensive_value,defensive_value,num_actions
568,Marcus Bontempelli,Western Bulldogs,105.587179,101.692887,3.894292,3813
795,Taylor Walker,Adelaide,101.812900,106.926823,-5.113923,1421
769,Shai Bolton,Richmond,97.526887,99.829774,-2.302887,2585
826,Tom Hawkins,Geelong,89.500502,97.503331,-8.002829,1831
134,Charlie Cameron,Brisbane Lions,89.247688,93.268298,-4.020611,1537


In [41]:
players_noshot_vaep = get_player_vaep(vaep_data[vaep_data['action_type'] != "Shot"])
players_noshot_vaep.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions
568,Marcus Bontempelli,Western Bulldogs,81.747753,77.704244,4.043509,3668
769,Shai Bolton,Richmond,80.677515,82.988352,-2.310837,2388
309,Jack Macrae,Western Bulldogs,76.027399,68.964089,7.063310,4355
831,Tom Liberatore,Western Bulldogs,72.661423,64.679532,7.981891,3379
145,Christian Petracca,Melbourne,72.260076,76.119954,-3.859877,3848
193,Darcy Moore,Collingwood,72.177033,22.768824,49.408209,2486
5,Aaron Naughton,Western Bulldogs,71.291354,75.932022,-4.640668,1360
147,Clayton Oliver,Melbourne,66.871767,63.966793,2.904974,3994
257,Harris Andrews,Brisbane Lions,65.469351,14.430758,51.038593,2822
682,Patrick Cripps,Carlton,65.130635,63.724909,1.405726,3105


Normalise for minutes played

In [43]:
player_minutes = get_player_minutes(player_stats)
players_vaep_mins = players_vaep.merge(player_minutes, how = 'left', on = "player")
player_ratings = convert_vaep_to_rating(players_vaep_mins)
player_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
795,Taylor Walker,Adelaide,101.812900,106.926823,-5.113923,1421,4318,51,2.357872,2.476304,-0.118433
138,Charlie Curnow,Carlton,83.383883,87.286268,-3.902386,1236,4015,44,2.076809,2.174004,-0.097195
769,Shai Bolton,Richmond,97.526887,99.829774,-2.302887,2585,5086,60,1.917556,1.962835,-0.045279
568,Marcus Bontempelli,Western Bulldogs,105.587179,101.692887,3.894292,3813,5575,66,1.893940,1.824088,0.069853
810,Toby Greene,Greater Western Sydney,76.196767,83.104904,-6.908137,1665,4383,48,1.738461,1.896074,-0.157612
829,Tom J. Lynch,Richmond,62.836297,67.856895,-5.020598,1048,3635,41,1.728646,1.866765,-0.138118
361,Jake Stringer,Essendon,67.826617,70.604125,-2.777509,1541,3957,50,1.714092,1.784284,-0.070192
50,Bayley Fritsch,Melbourne,89.059323,93.403534,-4.344211,1377,5212,62,1.708736,1.792086,-0.083350
541,Luke Breust,Hawthorn,73.486307,76.281684,-2.795378,1362,4304,55,1.707396,1.772344,-0.064948
884,Zac Bailey,Brisbane Lions,87.908824,88.858729,-0.949905,2405,5152,64,1.706305,1.724742,-0.018438


In [45]:
player_minutes = get_player_minutes(player_stats)
players_noshot_vaep_mins = players_noshot_vaep.merge(player_minutes, how = 'left', on = "player")
player_noshot_ratings = convert_vaep_to_rating(players_noshot_vaep_mins)
player_noshot_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
769,Shai Bolton,Richmond,80.677515,82.988352,-2.310837,2388,5086,60,1.586267,1.631702,-0.045435
233,Elliot Yeo,West Coast,23.795598,20.371657,3.423941,1037,1617,23,1.471589,1.259843,0.211746
568,Marcus Bontempelli,Western Bulldogs,81.747753,77.704244,4.043509,3668,5575,66,1.466327,1.393798,0.072529
138,Charlie Curnow,Carlton,58.226315,61.556127,-3.329812,1010,4015,44,1.450220,1.533154,-0.082934
683,Patrick Dangerfield,Geelong,47.008000,48.817873,-1.809873,2133,3272,47,1.436675,1.491989,-0.055314
831,Tom Liberatore,Western Bulldogs,72.661423,64.679532,7.981891,3379,5094,64,1.426412,1.269720,0.156692
193,Darcy Moore,Collingwood,72.177033,22.768824,49.408209,2486,5125,54,1.408332,0.444270,0.964063
55,Ben Cunnington,North Melbourne,25.670399,23.413761,2.256638,1216,1830,24,1.402754,1.279440,0.123314
309,Jack Macrae,Western Bulldogs,76.027399,68.964089,7.063310,4355,5421,67,1.402461,1.272165,0.130295
194,Darcy Parish,Essendon,57.856381,58.400769,-0.544388,3376,4212,52,1.373608,1.386533,-0.012925


In [46]:
player_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
795,Taylor Walker,Adelaide,101.812900,106.926823,-5.113923,1421,4318,51,2.357872,2.476304,-0.118433
138,Charlie Curnow,Carlton,83.383883,87.286268,-3.902386,1236,4015,44,2.076809,2.174004,-0.097195
769,Shai Bolton,Richmond,97.526887,99.829774,-2.302887,2585,5086,60,1.917556,1.962835,-0.045279
810,Toby Greene,Greater Western Sydney,76.196767,83.104904,-6.908137,1665,4383,48,1.738461,1.896074,-0.157612
829,Tom J. Lynch,Richmond,62.836297,67.856895,-5.020598,1048,3635,41,1.728646,1.866765,-0.138118
568,Marcus Bontempelli,Western Bulldogs,105.587179,101.692887,3.894292,3813,5575,66,1.893940,1.824088,0.069853
409,Jeremy Cameron,Geelong,84.098198,88.412294,-4.314097,1785,4931,55,1.705500,1.792989,-0.087489
50,Bayley Fritsch,Melbourne,89.059323,93.403534,-4.344211,1377,5212,62,1.708736,1.792086,-0.083350
361,Jake Stringer,Essendon,67.826617,70.604125,-2.777509,1541,3957,50,1.714092,1.784284,-0.070192
541,Luke Breust,Hawthorn,73.486307,76.281684,-2.795378,1362,4304,55,1.707396,1.772344,-0.064948


In [47]:
player_noshot_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
769,Shai Bolton,Richmond,80.677515,82.988352,-2.310837,2388,5086,60,1.586267,1.631702,-0.045435
138,Charlie Curnow,Carlton,58.226315,61.556127,-3.329812,1010,4015,44,1.450220,1.533154,-0.082934
683,Patrick Dangerfield,Geelong,47.008000,48.817873,-1.809873,2133,3272,47,1.436675,1.491989,-0.055314
829,Tom J. Lynch,Richmond,47.608532,52.027723,-4.419191,858,3635,41,1.309726,1.431299,-0.121573
568,Marcus Bontempelli,Western Bulldogs,81.747753,77.704244,4.043509,3668,5575,66,1.466327,1.393798,0.072529
194,Darcy Parish,Essendon,57.856381,58.400769,-0.544388,3376,4212,52,1.373608,1.386533,-0.012925
5,Aaron Naughton,Western Bulldogs,71.291354,75.932022,-4.640668,1360,5498,65,1.296678,1.381084,-0.084406
682,Patrick Cripps,Carlton,65.130635,63.724909,1.405726,3105,4776,58,1.363707,1.334274,0.029433
598,Max King,St Kilda,51.755185,54.861855,-3.106670,826,4211,49,1.229047,1.302822,-0.073775
145,Christian Petracca,Melbourne,72.260076,76.119954,-3.859877,3848,5853,67,1.234582,1.300529,-0.065947


In [48]:
player_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
757,Sam Taylor,Greater Western Sydney,62.401249,12.881392,49.519857,2277,4816,51,1.295707,0.267471,1.028236
193,Darcy Moore,Collingwood,72.866386,23.544922,49.321464,2495,5125,54,1.421783,0.459413,0.962370
816,Tom Barrass,West Coast,46.522266,7.041502,39.480764,2084,4501,49,1.033598,0.156443,0.877155
784,Steven May,Melbourne,59.095696,9.648768,49.446928,2918,5755,61,1.026858,0.167659,0.859199
216,Dougal Howard,St Kilda,49.579609,5.427478,44.152131,2183,5182,55,0.956766,0.104737,0.852029
257,Harris Andrews,Brisbane Lions,65.594618,14.622595,50.972023,2825,6217,64,1.055085,0.235203,0.819881
367,James Blanck,Hawthorn,14.204304,1.789206,12.415098,428,1535,19,0.925362,0.116561,0.808801
376,James Sicily,Hawthorn,38.649331,12.121820,26.527511,2231,3330,35,1.160641,0.364019,0.796622
735,Sam Collins,Gold Coast,55.104071,10.661419,44.442652,2237,5697,60,0.967247,0.187141,0.780106
23,Aliir Aliir,Port Adelaide,60.572481,14.777120,45.795361,2478,5871,61,1.031723,0.251697,0.780027


In [49]:
player_noshot_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
757,Sam Taylor,Greater Western Sydney,62.401249,12.881392,49.519857,2277,4816,51,1.295707,0.267471,1.028236
193,Darcy Moore,Collingwood,72.177033,22.768824,49.408209,2486,5125,54,1.408332,0.444270,0.964063
816,Tom Barrass,West Coast,45.851704,6.343986,39.507718,2083,4501,49,1.018700,0.140946,0.877754
784,Steven May,Melbourne,59.400753,9.974796,49.425957,2913,5755,61,1.032159,0.173324,0.858835
216,Dougal Howard,St Kilda,49.705025,5.569288,44.135736,2181,5182,55,0.959186,0.107474,0.851712
257,Harris Andrews,Brisbane Lions,65.469351,14.430758,51.038593,2822,6217,64,1.053070,0.232118,0.820952
367,James Blanck,Hawthorn,14.204304,1.789206,12.415098,428,1535,19,0.925362,0.116561,0.808801
376,James Sicily,Hawthorn,38.053308,11.512535,26.540773,2224,3330,35,1.142742,0.345722,0.797020
735,Sam Collins,Gold Coast,55.104071,10.661419,44.442652,2237,5697,60,0.967247,0.187141,0.780106
23,Aliir Aliir,Port Adelaide,61.522723,15.768616,45.754107,2474,5871,61,1.047909,0.268585,0.779324


VAEP by Action

In [61]:
action_vaep = get_vaep_action_summary(vaep_data)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-264.081620,-166.799853,-97.281766,2975,0.003139,-0.088767,-0.056067,-0.032700,-0.013357,-0.009944,-0.032462
5,Handball,-1061.198582,-496.097285,-565.101297,166573,0.175731,-0.006371,-0.002978,-0.003393,-0.053674,-0.029574,-0.188570
7,Kick,-300.460454,320.357662,-620.818116,220438,0.232557,-0.001363,0.001453,-0.002816,-0.015197,0.019098,-0.207163
0,Carry,2142.699031,1978.729543,163.969489,149555,0.157777,0.014327,0.013231,0.001096,0.108374,0.117960,0.054716
4,Gather,879.452630,791.904374,87.548256,37682,0.039754,0.023339,0.021015,0.002323,0.044481,0.047209,0.029214
11,Spoil,1053.226674,-198.786343,1252.013017,37977,0.040065,0.027733,-0.005234,0.032968,0.053270,-0.011850,0.417788
6,Hard Ball Get,1392.229069,866.809631,525.419439,48623,0.051296,0.028633,0.017827,0.010806,0.070417,0.051674,0.175329
13,Uncontested Mark,3026.849183,3062.745151,-35.895968,105273,0.111061,0.028752,0.029093,-0.000341,0.153093,0.182583,-0.011978
12,Tackle,10.682679,6.404204,4.278475,293,0.000309,0.036460,0.021857,0.014602,0.000540,0.000382,0.001428
9,Loose Ball Get,3944.134766,2994.892983,949.241783,100997,0.106550,0.039052,0.029653,0.009399,0.199488,0.178538,0.316756


In [62]:
match_data = get_match(vaep_data, match_id)
match_action_vaep = get_vaep_action_summary(match_data)
match_action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-0.127579,-0.133084,0.005505,3,0.001858,-0.042526,-0.044361,0.001835,-0.003514,-0.004080,0.001493
5,Handball,-1.774385,-0.528741,-1.245644,280,0.173375,-0.006337,-0.001888,-0.004449,-0.048878,-0.016212,-0.337826
7,Kick,-1.584179,-0.451818,-1.132361,374,0.231579,-0.004236,-0.001208,-0.003028,-0.043638,-0.013853,-0.307103
4,Gather,0.482535,0.610414,-0.127879,59,0.036533,0.008179,0.010346,-0.002167,0.013292,0.018716,-0.034682
11,Spoil,0.719012,-0.514802,1.233814,58,0.035913,0.012397,-0.008876,0.021273,0.019806,-0.015784,0.334618
0,Carry,4.920934,4.487724,0.433209,265,0.164087,0.018570,0.016935,0.001635,0.135554,0.137596,0.117489
6,Hard Ball Get,1.559284,1.464817,0.094467,80,0.049536,0.019491,0.018310,0.001181,0.042953,0.044912,0.025620
12,Uncontested Mark,4.924536,5.109785,-0.185249,193,0.119505,0.025516,0.026476,-0.000960,0.135653,0.156669,-0.050241
9,Loose Ball Get,6.340137,4.509594,1.830543,190,0.117647,0.033369,0.023735,0.009634,0.174648,0.138267,0.496454
8,Knock On,1.008632,1.027620,-0.018988,15,0.009288,0.067242,0.068508,-0.001266,0.027784,0.031507,-0.005150


In [64]:
player = "Christian Petracca"
player_data = get_player(vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-7.357052,-3.682931,-3.674121,880,0.217553,-0.008360,-0.004185,-0.004175,-0.084010,-0.040510,1.099965
2,Error,-2.635118,-1.577707,-1.057411,25,0.006180,-0.105405,-0.063108,-0.042296,-0.030090,-0.017354,0.316570
8,Knock On,0.130539,0.108344,0.022195,12,0.002967,0.010878,0.009029,0.001850,0.001491,0.001192,-0.006645
11,Spoil,0.212991,-0.000755,0.213746,17,0.004203,0.012529,-0.000044,0.012573,0.002432,-0.000008,-0.063992
1,Contested Mark,1.507972,1.471704,0.036267,17,0.004203,0.088704,0.086571,0.002133,0.017219,0.016188,-0.010858
7,Kick,5.547574,8.741902,-3.194328,838,0.207169,0.006620,0.010432,-0.003812,0.063348,0.096156,0.956324
0,Carry,8.565290,10.302913,-1.737623,622,0.153770,0.013771,0.016564,-0.002794,0.097807,0.113326,0.520213
6,Hard Ball Get,8.787554,8.172595,0.614959,298,0.073671,0.029488,0.027425,0.002064,0.100345,0.089894,-0.184108
3,Free For,9.306208,7.670144,1.636063,57,0.014091,0.163267,0.134564,0.028703,0.106267,0.084367,-0.489808
4,Gather,11.251435,9.398585,1.852850,309,0.076391,0.036412,0.030416,0.005996,0.128480,0.103379,-0.554710


In [65]:
player = "Nick Daicos"
player_data = get_player(vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-4.741945,-2.036085,-2.705860,554,0.213982,-0.008559,-0.003675,-0.004884,-0.113417,-0.056212,-0.484208
6,Kick,-0.697174,0.270212,-0.967385,589,0.227501,-0.001184,0.000459,-0.001642,-0.016675,0.007460,-0.173112
1,Error,-0.624791,-0.452724,-0.172068,7,0.002704,-0.089256,-0.064675,-0.024581,-0.014944,-0.012499,-0.030791
10,Spoil,-0.191354,0.047059,-0.238413,8,0.003090,-0.023919,0.005882,-0.029802,-0.004577,0.001299,-0.042663
7,Knock On,-0.089768,-0.097443,0.007675,6,0.002317,-0.014961,-0.016240,0.001279,-0.002147,-0.002690,0.001373
5,Hard Ball Get,1.857238,1.237932,0.619306,77,0.029741,0.024120,0.016077,0.008043,0.044421,0.034177,0.110823
3,Gather,2.052703,1.675406,0.377297,93,0.035921,0.022072,0.018015,0.004057,0.049096,0.046254,0.067516
11,Uncontested Mark,3.671499,3.851685,-0.180185,183,0.070684,0.020063,0.021047,-0.000985,0.087814,0.106337,-0.032244
2,Free For,6.352579,4.087389,2.265191,50,0.019312,0.127052,0.081748,0.045304,0.151940,0.112844,0.405351
9,Shot,9.038062,9.146199,-0.108138,51,0.019699,0.177217,0.179337,-0.002120,0.216171,0.252506,-0.019351


In [67]:
player = "Darcy Moore"
player_data = get_player(vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
7,Kick,-2.307512,-1.237706,-1.069807,586,0.234870,-0.003938,-0.002112,-0.001826,-0.031668,-0.052568,-0.021690
2,Error,-0.821016,-0.130488,-0.690529,6,0.002405,-0.136836,-0.021748,-0.115088,-0.011267,-0.005542,-0.014001
10,Shot,0.689353,0.776098,-0.086745,9,0.003607,0.076595,0.086233,-0.009638,0.009461,0.032962,-0.001759
4,Gather,1.126167,1.331274,-0.205107,44,0.017635,0.025595,0.030256,-0.004662,0.015455,0.056542,-0.004159
5,Handball,2.218869,2.435414,-0.216545,275,0.110220,0.008069,0.008856,-0.000787,0.030451,0.103437,-0.004390
8,Knock On,2.540346,1.073815,1.466531,59,0.023647,0.043057,0.018200,0.024856,0.034863,0.045607,0.029734
6,Hard Ball Get,3.842746,0.591300,3.251445,98,0.039279,0.039212,0.006034,0.033178,0.052737,0.025114,0.065924
12,Uncontested Mark,6.484589,5.650833,0.833756,346,0.138677,0.018742,0.016332,0.002410,0.088993,0.240002,0.016905
9,Loose Ball Get,6.718773,2.096851,4.621922,192,0.076954,0.034994,0.010921,0.024073,0.092207,0.089057,0.093710
3,Free For,6.828060,3.258172,3.569888,40,0.016032,0.170702,0.081454,0.089247,0.093707,0.138381,0.072380


In [70]:
player = "Taylor Walker"
player_data = get_player(vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value").round(2)

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-4.55,-3.20,-1.35,199,0.14,-0.02,-0.02,-0.01,-0.04,-0.03,0.26
11,Spoil,-1.61,-1.28,-0.32,33,0.02,-0.05,-0.04,-0.01,-0.02,-0.01,0.06
2,Error,-0.67,-0.52,-0.16,7,0.00,-0.10,-0.07,-0.02,-0.01,-0.00,0.03
4,Gather,0.31,0.50,-0.19,40,0.03,0.01,0.01,-0.00,0.00,0.00,0.04
0,Carry,0.76,1.11,-0.35,104,0.07,0.01,0.01,-0.00,0.01,0.01,0.07
8,Knock On,1.22,1.25,-0.03,18,0.01,0.07,0.07,-0.00,0.01,0.01,0.01
7,Kick,3.54,4.42,-0.87,215,0.15,0.02,0.02,-0.00,0.03,0.04,0.17
6,Hard Ball Get,3.78,4.28,-0.50,108,0.08,0.04,0.04,-0.00,0.04,0.04,0.10
9,Loose Ball Get,7.33,7.81,-0.48,116,0.08,0.06,0.07,-0.00,0.07,0.07,0.09
3,Free For,13.17,12.86,0.31,56,0.04,0.24,0.23,0.01,0.13,0.12,-0.06


By Season

In [71]:
players_2021 = get_player_vaep(vaep_2021)
players_2022 = get_player_vaep(vaep_2022)
players_2023 = get_player_vaep(vaep_2023)

In [72]:
player_minutes_2021 = get_player_minutes(player_stats_2021)
player_minutes_2022 = get_player_minutes(player_stats_2022)
player_minutes_2023 = get_player_minutes(player_stats_2023)

In [73]:
players_2021 = players_2021.merge(player_minutes_2021, how = "left", on = "player")
players_2022 = players_2022.merge(player_minutes_2022, how = "left", on = "player")
players_2023 = players_2023.merge(player_minutes_2023, how = "left", on = "player")

In [74]:
player_ratings_2021 = convert_vaep_to_rating(players_2021)
player_ratings_2022 = convert_vaep_to_rating(players_2022)
player_ratings_2023 = convert_vaep_to_rating(players_2023)

2021

In [75]:
player_ratings_2021.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
420,Marcus Bontempelli,Western Bulldogs,41.089256,40.479472,0.609784,1569,2193,26,1.873655,1.845849,0.027806
232,Jack Macrae,Western Bulldogs,38.953811,35.768880,3.184931,1872,2154,26,1.808441,1.660579,0.147861
37,Bayley Fritsch,Melbourne,37.731554,39.321782,-1.590228,523,2049,24,1.841462,1.919072,-0.077610
268,Jake Stringer,Essendon,36.223728,36.838667,-0.614940,661,1526,19,2.373770,2.414067,-0.040297
110,Christian Petracca,Melbourne,34.525230,36.433127,-1.907897,1524,2237,25,1.543372,1.628660,-0.085288
100,Charlie Cameron,Brisbane Lions,34.504675,36.566087,-2.061412,560,2082,24,1.657285,1.756296,-0.099011
112,Clayton Oliver,Melbourne,34.315764,32.301553,2.014210,1702,2211,25,1.552047,1.460948,0.091100
154,Dayne Zorko,Brisbane Lions,34.129660,32.432889,1.696770,1199,1900,23,1.796298,1.706994,0.089304
381,Lance Franklin,Sydney,33.781166,35.415754,-1.634589,459,1615,18,2.091713,2.192926,-0.101213
598,Toby Greene,Greater Western Sydney,32.740667,34.476528,-1.735862,663,1627,18,2.012334,2.119024,-0.106691


In [76]:
player_ratings_2021.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
268,Jake Stringer,Essendon,36.223728,36.838667,-0.614940,661,1526,19,2.373770,2.414067,-0.040297
305,Jeremy Cameron,Geelong,26.239216,27.265081,-1.025866,386,1237,15,2.121198,2.204129,-0.082932
589,Taylor Walker,Adelaide,28.723013,30.141224,-1.418211,441,1369,16,2.098102,2.201696,-0.103595
381,Lance Franklin,Sydney,33.781166,35.415754,-1.634589,459,1615,18,2.091713,2.192926,-0.101213
598,Toby Greene,Greater Western Sydney,32.740667,34.476528,-1.735862,663,1627,18,2.012334,2.119024,-0.106691
420,Marcus Bontempelli,Western Bulldogs,41.089256,40.479472,0.609784,1569,2193,26,1.873655,1.845849,0.027806
37,Bayley Fritsch,Melbourne,37.731554,39.321782,-1.590228,523,2049,24,1.841462,1.919072,-0.077610
232,Jack Macrae,Western Bulldogs,38.953811,35.768880,3.184931,1872,2154,26,1.808441,1.660579,0.147861
503,Patrick Dangerfield,Geelong,19.504833,19.533009,-0.028177,749,1080,15,1.806003,1.808612,-0.002609
154,Dayne Zorko,Brisbane Lions,34.129660,32.432889,1.696770,1199,1900,23,1.796298,1.706994,0.089304


In [77]:
player_ratings_2021.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
420,Marcus Bontempelli,Western Bulldogs,41.089256,40.479472,0.609784,1569,2193,26,1.873655,1.845849,0.027806
37,Bayley Fritsch,Melbourne,37.731554,39.321782,-1.590228,523,2049,24,1.841462,1.919072,-0.077610
268,Jake Stringer,Essendon,36.223728,36.838667,-0.614940,661,1526,19,2.373770,2.414067,-0.040297
100,Charlie Cameron,Brisbane Lions,34.504675,36.566087,-2.061412,560,2082,24,1.657285,1.756296,-0.099011
110,Christian Petracca,Melbourne,34.525230,36.433127,-1.907897,1524,2237,25,1.543372,1.628660,-0.085288
232,Jack Macrae,Western Bulldogs,38.953811,35.768880,3.184931,1872,2154,26,1.808441,1.660579,0.147861
381,Lance Franklin,Sydney,33.781166,35.415754,-1.634589,459,1615,18,2.091713,2.192926,-0.101213
104,Charlie Dixon,Port Adelaide,32.721496,34.652139,-1.930643,675,2276,24,1.437676,1.522502,-0.084826
598,Toby Greene,Greater Western Sydney,32.740667,34.476528,-1.735862,663,1627,18,2.012334,2.119024,-0.106691
623,Tom Papley,Sydney,31.696339,33.113764,-1.417426,660,2008,23,1.578503,1.649092,-0.070589


In [78]:
player_ratings_2021.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
268,Jake Stringer,Essendon,36.223728,36.838667,-0.614940,661,1526,19,2.373770,2.414067,-0.040297
305,Jeremy Cameron,Geelong,26.239216,27.265081,-1.025866,386,1237,15,2.121198,2.204129,-0.082932
589,Taylor Walker,Adelaide,28.723013,30.141224,-1.418211,441,1369,16,2.098102,2.201696,-0.103595
381,Lance Franklin,Sydney,33.781166,35.415754,-1.634589,459,1615,18,2.091713,2.192926,-0.101213
598,Toby Greene,Greater Western Sydney,32.740667,34.476528,-1.735862,663,1627,18,2.012334,2.119024,-0.106691
37,Bayley Fritsch,Melbourne,37.731554,39.321782,-1.590228,523,2049,24,1.841462,1.919072,-0.077610
420,Marcus Bontempelli,Western Bulldogs,41.089256,40.479472,0.609784,1569,2193,26,1.873655,1.845849,0.027806
503,Patrick Dangerfield,Geelong,19.504833,19.533009,-0.028177,749,1080,15,1.806003,1.808612,-0.002609
401,Luke Breust,Hawthorn,25.381825,26.957913,-1.576088,521,1508,19,1.683145,1.787660,-0.104515
200,Harry McKay,Carlton,27.506344,29.210159,-1.703815,405,1637,19,1.680290,1.784371,-0.104082


In [79]:
player_ratings_2021.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
163,Dougal Howard,St Kilda,23.506703,1.209586,22.297116,935,1983,21,1.185411,0.060998,1.124413
194,Harris Andrews,Brisbane Lions,30.210002,8.824409,21.385593,1109,2224,23,1.358363,0.396781,0.961582
617,Tom Jonas,Port Adelaide,23.950898,5.498355,18.452543,985,2297,24,1.042703,0.239371,0.803332
18,Aliir Aliir,Port Adelaide,24.692692,6.783334,17.909358,1042,2315,24,1.066639,0.293017,0.773622
580,Steven May,Melbourne,19.638923,2.120272,17.518651,1052,2145,23,0.915568,0.098847,0.816720
557,Sam Taylor,Greater Western Sydney,21.844168,4.470603,17.373565,829,1781,19,1.226511,0.251016,0.975495
253,Jacob Weitering,Carlton,21.166083,4.562356,16.603727,1063,2110,22,1.003132,0.216225,0.786906
324,Jordan Ridley,Essendon,19.340771,3.054141,16.286630,1183,2008,22,0.963186,0.152099,0.811087
391,Liam Jones,Carlton,21.747704,5.900750,15.846955,716,1814,19,1.198881,0.325289,0.873592
540,Sam Collins,Gold Coast,18.518125,3.089698,15.428427,725,1788,20,1.035689,0.172802,0.862887


In [80]:
player_ratings_2021.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
163,Dougal Howard,St Kilda,23.506703,1.209586,22.297116,935,1983,21,1.185411,0.060998,1.124413
557,Sam Taylor,Greater Western Sydney,21.844168,4.470603,17.373565,829,1781,19,1.226511,0.251016,0.975495
194,Harris Andrews,Brisbane Lions,30.210002,8.824409,21.385593,1109,2224,23,1.358363,0.396781,0.961582
391,Liam Jones,Carlton,21.747704,5.900750,15.846955,716,1814,19,1.198881,0.325289,0.873592
540,Sam Collins,Gold Coast,18.518125,3.089698,15.428427,725,1788,20,1.035689,0.172802,0.862887
580,Steven May,Melbourne,19.638923,2.120272,17.518651,1052,2145,23,0.915568,0.098847,0.816720
324,Jordan Ridley,Essendon,19.340771,3.054141,16.286630,1183,2008,22,0.963186,0.152099,0.811087
617,Tom Jonas,Port Adelaide,23.950898,5.498355,18.452543,985,2297,24,1.042703,0.239371,0.803332
144,Darcy Moore,Collingwood,16.770282,7.077775,9.692507,599,1211,13,1.384829,0.584457,0.800372
253,Jacob Weitering,Carlton,21.166083,4.562356,16.603727,1063,2110,22,1.003132,0.216225,0.786906


2022

In [81]:
player_ratings_2022.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,42.300445,44.370708,-2.070264,903,1985,23,2.131005,2.235300,-0.104295
104,Charlie Curnow,Carlton,40.769662,42.632288,-1.862626,574,2022,22,2.016304,2.108422,-0.092118
626,Tom J. Lynch,Richmond,37.523998,40.127428,-2.603430,514,1601,19,2.343785,2.506398,-0.162613
650,Tyson Stengle,Geelong,37.202625,38.653574,-1.450949,781,2059,25,1.806830,1.877298,-0.070469
599,Taylor Walker,Adelaide,37.015581,39.006983,-1.991402,553,1566,18,2.363702,2.490867,-0.127165
211,Isaac Heeney,Sydney,36.353880,36.938159,-0.584279,896,2136,25,1.701961,1.729315,-0.027354
623,Tom Hawkins,Geelong,35.995964,38.668503,-2.672539,696,2366,25,1.521385,1.634341,-0.112956
311,Jeremy Cameron,Geelong,35.540523,37.239889,-1.699366,833,2214,24,1.605263,1.682018,-0.076755
432,Marcus Bontempelli,Western Bulldogs,35.538227,34.512212,1.026015,1158,1863,22,1.907581,1.852507,0.055073
38,Bayley Fritsch,Melbourne,35.130935,36.677769,-1.546834,514,2009,24,1.748678,1.825673,-0.076995


In [82]:
player_ratings_2022.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
599,Taylor Walker,Adelaide,37.015581,39.006983,-1.991402,553,1566,18,2.363702,2.490867,-0.127165
626,Tom J. Lynch,Richmond,37.523998,40.127428,-2.603430,514,1601,19,2.343785,2.506398,-0.162613
582,Shai Bolton,Richmond,42.300445,44.370708,-2.070264,903,1985,23,2.131005,2.235300,-0.104295
104,Charlie Curnow,Carlton,40.769662,42.632288,-1.862626,574,2022,22,2.016304,2.108422,-0.092118
432,Marcus Bontempelli,Western Bulldogs,35.538227,34.512212,1.026015,1158,1863,22,1.907581,1.852507,0.055073
514,Patrick Cripps,Carlton,31.884397,32.661767,-0.777370,1252,1707,21,1.867862,1.913402,-0.045540
411,Luke Breust,Hawthorn,30.320850,31.227694,-0.906844,509,1642,21,1.846580,1.901808,-0.055228
650,Tyson Stengle,Geelong,37.202625,38.653574,-1.450949,781,2059,25,1.806830,1.877298,-0.070469
666,Zac Bailey,Brisbane Lions,34.295735,34.826140,-0.530405,882,1903,24,1.802193,1.830065,-0.027872
97,Chad Warner,Sydney,34.100716,34.233588,-0.132872,1208,1938,24,1.759583,1.766439,-0.006856


In [83]:
player_ratings_2022.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,42.300445,44.370708,-2.070264,903,1985,23,2.131005,2.235300,-0.104295
104,Charlie Curnow,Carlton,40.769662,42.632288,-1.862626,574,2022,22,2.016304,2.108422,-0.092118
626,Tom J. Lynch,Richmond,37.523998,40.127428,-2.603430,514,1601,19,2.343785,2.506398,-0.162613
599,Taylor Walker,Adelaide,37.015581,39.006983,-1.991402,553,1566,18,2.363702,2.490867,-0.127165
623,Tom Hawkins,Geelong,35.995964,38.668503,-2.672539,696,2366,25,1.521385,1.634341,-0.112956
650,Tyson Stengle,Geelong,37.202625,38.653574,-1.450949,781,2059,25,1.806830,1.877298,-0.070469
311,Jeremy Cameron,Geelong,35.540523,37.239889,-1.699366,833,2214,24,1.605263,1.682018,-0.076755
211,Isaac Heeney,Sydney,36.353880,36.938159,-0.584279,896,2136,25,1.701961,1.729315,-0.027354
38,Bayley Fritsch,Melbourne,35.130935,36.677769,-1.546834,514,2009,24,1.748678,1.825673,-0.076995
666,Zac Bailey,Brisbane Lions,34.295735,34.826140,-0.530405,882,1903,24,1.802193,1.830065,-0.027872


In [84]:
player_ratings_2022.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,37.523998,40.127428,-2.603430,514,1601,19,2.343785,2.506398,-0.162613
599,Taylor Walker,Adelaide,37.015581,39.006983,-1.991402,553,1566,18,2.363702,2.490867,-0.127165
582,Shai Bolton,Richmond,42.300445,44.370708,-2.070264,903,1985,23,2.131005,2.235300,-0.104295
104,Charlie Curnow,Carlton,40.769662,42.632288,-1.862626,574,2022,22,2.016304,2.108422,-0.092118
514,Patrick Cripps,Carlton,31.884397,32.661767,-0.777370,1252,1707,21,1.867862,1.913402,-0.045540
411,Luke Breust,Hawthorn,30.320850,31.227694,-0.906844,509,1642,21,1.846580,1.901808,-0.055228
650,Tyson Stengle,Geelong,37.202625,38.653574,-1.450949,781,2059,25,1.806830,1.877298,-0.070469
432,Marcus Bontempelli,Western Bulldogs,35.538227,34.512212,1.026015,1158,1863,22,1.907581,1.852507,0.055073
666,Zac Bailey,Brisbane Lions,34.295735,34.826140,-0.530405,882,1903,24,1.802193,1.830065,-0.027872
38,Bayley Fritsch,Melbourne,35.130935,36.677769,-1.546834,514,2009,24,1.748678,1.825673,-0.076995


In [85]:
player_ratings_2022.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
144,Darcy Moore,Collingwood,33.758473,9.662224,24.096249,1044,2314,24,1.458880,0.417555,1.041324
592,Steven May,Melbourne,26.122164,4.847935,21.274229,1106,2049,22,1.274874,0.236600,1.038274
573,Sam Taylor,Greater Western Sydney,25.759106,6.012629,19.746478,1018,2132,22,1.208213,0.282018,0.926195
286,James Sicily,Hawthorn,25.453052,7.095073,18.357979,1363,2110,22,1.206306,0.336259,0.870046
313,Jeremy Howe,Collingwood,25.026927,6.999356,18.027572,1051,2214,24,1.130394,0.316141,0.814253
188,Harris Andrews,Brisbane Lions,19.730447,2.031744,17.698703,948,2337,24,0.844264,0.086938,0.757326
615,Tom Barrass,West Coast,21.741598,4.050955,17.690643,865,1767,19,1.230424,0.229256,1.001168
555,Sam Collins,Gold Coast,22.224864,4.693080,17.531784,798,2154,22,1.031795,0.217877,0.813918
73,Brennan Cox,Fremantle,22.104377,5.650903,16.453474,1024,2000,23,1.105219,0.282545,0.822674
331,Jordan Ridley,Essendon,16.384198,1.779026,14.605172,968,1794,20,0.913278,0.099165,0.814112


In [86]:
player_ratings_2022.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
144,Darcy Moore,Collingwood,33.758473,9.662224,24.096249,1044,2314,24,1.458880,0.417555,1.041324
592,Steven May,Melbourne,26.122164,4.847935,21.274229,1106,2049,22,1.274874,0.236600,1.038274
615,Tom Barrass,West Coast,21.741598,4.050955,17.690643,865,1767,19,1.230424,0.229256,1.001168
573,Sam Taylor,Greater Western Sydney,25.759106,6.012629,19.746478,1018,2132,22,1.208213,0.282018,0.926195
286,James Sicily,Hawthorn,25.453052,7.095073,18.357979,1363,2110,22,1.206306,0.336259,0.870046
73,Brennan Cox,Fremantle,22.104377,5.650903,16.453474,1024,2000,23,1.105219,0.282545,0.822674
313,Jeremy Howe,Collingwood,25.026927,6.999356,18.027572,1051,2214,24,1.130394,0.316141,0.814253
331,Jordan Ridley,Essendon,16.384198,1.779026,14.605172,968,1794,20,0.913278,0.099165,0.814112
555,Sam Collins,Gold Coast,22.224864,4.693080,17.531784,798,2154,22,1.031795,0.217877,0.813918
188,Harris Andrews,Brisbane Lions,19.730447,2.031744,17.698703,948,2337,24,0.844264,0.086938,0.757326


2023

In [89]:
player_ratings_2023.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
100,Charlie Curnow,Carlton,39.268889,40.808032,-1.539143,557,1677,18,2.341615,2.433395,-0.091780
554,Taylor Walker,Adelaide,36.074306,37.778616,-1.704310,427,1383,17,2.608410,2.731642,-0.123233
401,Marcus Bontempelli,Western Bulldogs,28.959696,26.701203,2.258493,1086,1519,18,1.906497,1.757815,0.148683
543,Shai Bolton,Richmond,27.920474,27.750210,0.170264,835,1473,17,1.895484,1.883925,0.011559
622,Zak Butters,Port Adelaide,27.098623,25.842744,1.255878,1056,1415,18,1.915097,1.826342,0.088755
478,Oscar Allen,West Coast,27.074040,26.930926,0.143113,413,1609,18,1.682663,1.673768,0.008895
461,Nick Larkey,North Melbourne,26.490235,28.354192,-1.863957,352,1628,17,1.627164,1.741658,-0.114494
111,Connor Rozee,Port Adelaide,26.108694,26.993111,-0.884416,988,1509,18,1.730198,1.788808,-0.058609
301,Joe Daniher,Brisbane Lions,25.920428,27.052174,-1.131747,537,1496,17,1.732649,1.808300,-0.075652
465,Noah Anderson,Gold Coast,24.941973,24.462025,0.479948,1075,1525,18,1.635539,1.604067,0.031472


In [90]:
player_ratings_2023.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
554,Taylor Walker,Adelaide,36.074306,37.778616,-1.704310,427,1383,17,2.608410,2.731642,-0.123233
100,Charlie Curnow,Carlton,39.268889,40.808032,-1.539143,557,1677,18,2.341615,2.433395,-0.091780
622,Zak Butters,Port Adelaide,27.098623,25.842744,1.255878,1056,1415,18,1.915097,1.826342,0.088755
401,Marcus Bontempelli,Western Bulldogs,28.959696,26.701203,2.258493,1086,1519,18,1.906497,1.757815,0.148683
543,Shai Bolton,Richmond,27.920474,27.750210,0.170264,835,1473,17,1.895484,1.883925,0.011559
4,Adam Cerra,Carlton,23.528473,21.549870,1.978604,1006,1304,17,1.804331,1.652597,0.151733
301,Joe Daniher,Brisbane Lions,25.920428,27.052174,-1.131747,537,1496,17,1.732649,1.808300,-0.075652
111,Connor Rozee,Port Adelaide,26.108694,26.993111,-0.884416,988,1509,18,1.730198,1.788808,-0.058609
108,Cody Weightman,Western Bulldogs,18.525636,18.758728,-0.233092,318,1072,14,1.728138,1.749881,-0.021744
616,Zac Bailey,Brisbane Lions,22.205247,23.038767,-0.833520,586,1296,16,1.713368,1.777683,-0.064315


In [91]:
player_ratings_2023.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
100,Charlie Curnow,Carlton,39.268889,40.808032,-1.539143,557,1677,18,2.341615,2.433395,-0.091780
554,Taylor Walker,Adelaide,36.074306,37.778616,-1.704310,427,1383,17,2.608410,2.731642,-0.123233
461,Nick Larkey,North Melbourne,26.490235,28.354192,-1.863957,352,1628,17,1.627164,1.741658,-0.114494
543,Shai Bolton,Richmond,27.920474,27.750210,0.170264,835,1473,17,1.895484,1.883925,0.011559
301,Joe Daniher,Brisbane Lions,25.920428,27.052174,-1.131747,537,1496,17,1.732649,1.808300,-0.075652
111,Connor Rozee,Port Adelaide,26.108694,26.993111,-0.884416,988,1509,18,1.730198,1.788808,-0.058609
478,Oscar Allen,West Coast,27.074040,26.930926,0.143113,413,1609,18,1.682663,1.673768,0.008895
401,Marcus Bontempelli,Western Bulldogs,28.959696,26.701203,2.258493,1086,1519,18,1.906497,1.757815,0.148683
575,Tom Hawkins,Geelong,23.755181,25.864499,-2.109318,426,1666,18,1.425881,1.552491,-0.126610
622,Zak Butters,Port Adelaide,27.098623,25.842744,1.255878,1056,1415,18,1.915097,1.826342,0.088755


In [92]:
player_ratings_2023.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
554,Taylor Walker,Adelaide,36.074306,37.778616,-1.704310,427,1383,17,2.608410,2.731642,-0.123233
100,Charlie Curnow,Carlton,39.268889,40.808032,-1.539143,557,1677,18,2.341615,2.433395,-0.091780
543,Shai Bolton,Richmond,27.920474,27.750210,0.170264,835,1473,17,1.895484,1.883925,0.011559
622,Zak Butters,Port Adelaide,27.098623,25.842744,1.255878,1056,1415,18,1.915097,1.826342,0.088755
301,Joe Daniher,Brisbane Lions,25.920428,27.052174,-1.131747,537,1496,17,1.732649,1.808300,-0.075652
111,Connor Rozee,Port Adelaide,26.108694,26.993111,-0.884416,988,1509,18,1.730198,1.788808,-0.058609
616,Zac Bailey,Brisbane Lions,22.205247,23.038767,-0.833520,586,1296,16,1.713368,1.777683,-0.064315
563,Toby Greene,Greater Western Sydney,21.271338,24.142042,-2.870704,557,1365,15,1.558340,1.768648,-0.210308
401,Marcus Bontempelli,Western Bulldogs,28.959696,26.701203,2.258493,1086,1519,18,1.906497,1.757815,0.148683
108,Cody Weightman,Western Bulldogs,18.525636,18.758728,-0.233092,318,1072,14,1.728138,1.749881,-0.021744


In [93]:
player_ratings_2023.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
140,Darcy Moore,Collingwood,22.337631,6.804922,15.532709,852,1600,17,1.396102,0.425308,0.970794
17,Aliir Aliir,Port Adelaide,20.485109,5.015150,15.469959,680,1759,18,1.164588,0.285114,0.879475
96,Charlie Ballard,Gold Coast,18.133697,3.956896,14.176801,714,1717,18,1.056127,0.230454,0.825673
214,Jack Buckley,Greater Western Sydney,19.727666,6.618771,13.108895,701,1592,17,1.239175,0.415752,0.823423
14,Alex Pearce,Fremantle,16.126837,3.364238,12.762599,537,1694,18,0.951997,0.198597,0.753400
182,Harris Andrews,Brisbane Lions,15.654169,3.766442,11.887727,768,1656,17,0.945300,0.227442,0.717858
462,Nick Murray,Adelaide,15.718699,3.889466,11.829233,520,1353,17,1.161766,0.287470,0.874297
569,Tom Barrass,West Coast,13.302724,1.591095,11.711629,572,1341,14,0.992000,0.118650,0.873350
517,Sam Collins,Gold Coast,14.361082,2.878642,11.482441,714,1755,18,0.818295,0.164025,0.654270
394,Luke Ryan,Fremantle,13.954033,2.607004,11.347029,1051,1534,18,0.909650,0.169948,0.739702


In [94]:
player_ratings_2023.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
140,Darcy Moore,Collingwood,22.337631,6.804922,15.532709,852,1600,17,1.396102,0.425308,0.970794
17,Aliir Aliir,Port Adelaide,20.485109,5.015150,15.469959,680,1759,18,1.164588,0.285114,0.879475
462,Nick Murray,Adelaide,15.718699,3.889466,11.829233,520,1353,17,1.161766,0.287470,0.874297
569,Tom Barrass,West Coast,13.302724,1.591095,11.711629,572,1341,14,0.992000,0.118650,0.873350
96,Charlie Ballard,Gold Coast,18.133697,3.956896,14.176801,714,1717,18,1.056127,0.230454,0.825673
214,Jack Buckley,Greater Western Sydney,19.727666,6.618771,13.108895,701,1592,17,1.239175,0.415752,0.823423
14,Alex Pearce,Fremantle,16.126837,3.364238,12.762599,537,1694,18,0.951997,0.198597,0.753400
394,Luke Ryan,Fremantle,13.954033,2.607004,11.347029,1051,1534,18,0.909650,0.169948,0.739702
70,Brennan Cox,Fremantle,16.555768,5.955624,10.600144,789,1442,16,1.148112,0.413011,0.735100
463,Nick Vlastuin,Richmond,15.287703,5.614641,9.673062,737,1326,15,1.152919,0.423427,0.729492


In [99]:
action_vaep = get_vaep_action_summary(vaep_2021)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-87.733365,-54.728839,-33.004526,972,0.002816,-0.090261,-0.056305,-0.033955,-0.012567,-0.009260,-0.030826
5,Handball,-336.323878,-136.211799,-200.112079,60799,0.176132,-0.005532,-0.002240,-0.003291,-0.048177,-0.023046,-0.186904
7,Kick,-117.958843,90.784690,-208.743533,80643,0.233619,-0.001463,0.001126,-0.002588,-0.016897,0.015360,-0.194966
0,Carry,736.376188,690.565323,45.810865,54634,0.158272,0.013478,0.012640,0.000839,0.105482,0.116839,0.042787
4,Gather,321.380563,291.453935,29.926628,14029,0.040641,0.022908,0.020775,0.002133,0.046036,0.049312,0.027951
11,Spoil,363.947174,-71.805942,435.753116,13590,0.039370,0.026781,-0.005284,0.032064,0.052133,-0.012149,0.406992
13,Uncontested Mark,1046.312638,1064.049967,-17.737330,38669,0.112022,0.027058,0.027517,-0.000459,0.149879,0.180030,-0.016567
6,Hard Ball Get,540.202811,335.433821,204.768990,18947,0.054889,0.028511,0.017704,0.010807,0.077381,0.056753,0.191253
9,Loose Ball Get,1327.010388,1002.894850,324.115538,34567,0.100139,0.038390,0.029013,0.009376,0.190087,0.169683,0.302723
12,Tackle,5.159365,3.343083,1.816282,129,0.000374,0.039995,0.025915,0.014080,0.000739,0.000566,0.001696


In [100]:
action_vaep = get_vaep_action_summary(vaep_2022)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-103.790407,-64.254386,-39.536021,1198,0.003503,-0.086636,-0.053635,-0.033002,-0.014279,-0.010397,-0.036321
5,Handball,-383.880038,-180.594300,-203.285738,59270,0.173288,-0.006477,-0.003047,-0.003430,-0.052813,-0.029222,-0.186757
7,Kick,-88.047384,140.921500,-228.968884,79965,0.233794,-0.001101,0.001762,-0.002863,-0.012113,0.022802,-0.210352
0,Carry,760.325295,699.595952,60.729343,53272,0.155752,0.014273,0.013133,0.001140,0.104604,0.113201,0.055792
4,Gather,317.858734,284.679159,33.179575,13390,0.039148,0.023739,0.021261,0.002478,0.043730,0.046064,0.030482
11,Spoil,392.534377,-73.915415,466.449792,13964,0.040827,0.028110,-0.005293,0.033404,0.054004,-0.011960,0.428524
6,Hard Ball Get,487.953514,306.729962,181.223552,17178,0.050223,0.028406,0.017856,0.010550,0.067131,0.049632,0.166489
13,Uncontested Mark,1102.155363,1116.298165,-14.142802,38101,0.111396,0.028927,0.029298,-0.000371,0.151632,0.180627,-0.012993
9,Loose Ball Get,1428.541991,1095.052047,333.489944,37170,0.108674,0.038433,0.029461,0.008972,0.196535,0.177189,0.306375
12,Tackle,3.391780,1.892562,1.499218,84,0.000246,0.040378,0.022531,0.017848,0.000467,0.000306,0.001377


In [97]:
action_vaep = get_vaep_action_summary(vaep_2023)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-72.557848,-47.816628,-24.741220,805,0.003088,-0.090134,-0.059400,-0.030734,-0.013141,-0.010208,-0.029538
5,Handball,-340.994666,-179.291185,-161.703481,46504,0.178405,-0.007333,-0.003855,-0.003477,-0.061756,-0.038277,-0.193057
7,Kick,-94.454227,88.651473,-183.105699,59830,0.229528,-0.001579,0.001482,-0.003060,-0.017106,0.018926,-0.218609
0,Carry,645.997548,588.568268,57.429281,41649,0.159780,0.015511,0.014132,0.001379,0.116994,0.125655,0.068565
4,Gather,240.213333,215.771280,24.442053,10263,0.039372,0.023406,0.021024,0.002382,0.043504,0.046065,0.029181
12,Tackle,2.131533,1.168558,0.962975,80,0.000307,0.026644,0.014607,0.012037,0.000386,0.000249,0.001150
11,Spoil,296.745123,-53.064987,349.810110,10423,0.039986,0.028470,-0.005091,0.033561,0.053743,-0.011329,0.417637
6,Hard Ball Get,364.072745,224.645848,139.426897,12498,0.047947,0.029130,0.017975,0.011156,0.065936,0.047960,0.166461
13,Uncontested Mark,878.381183,882.397019,-4.015836,28503,0.109347,0.030817,0.030958,-0.000141,0.159081,0.188385,-0.004794
9,Loose Ball Get,1188.582387,896.946086,291.636301,29260,0.112251,0.040621,0.030654,0.009967,0.215260,0.191491,0.348184


In [101]:
player = "Nick Daicos"
player_data = get_player(vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-2.793233,-0.902740,-1.890493,283,0.198457,-0.009870,-0.003190,-0.006680,-0.157578,-0.060937,-0.649301
6,Kick,-0.944299,-0.086025,-0.858274,344,0.241234,-0.002745,-0.000250,-0.002495,-0.053272,-0.005807,-0.294780
1,Error,-0.527283,-0.370721,-0.156562,6,0.004208,-0.087880,-0.061787,-0.026094,-0.029746,-0.025024,-0.053772
10,Spoil,-0.184257,-0.020772,-0.163485,5,0.003506,-0.036851,-0.004154,-0.032697,-0.010395,-0.001402,-0.056150
7,Knock On,0.020363,0.011416,0.008947,4,0.002805,0.005091,0.002854,0.002237,0.001149,0.000771,0.003073
3,Gather,0.555253,0.446429,0.108824,44,0.030856,0.012619,0.010146,0.002473,0.031324,0.030135,0.037376
5,Hard Ball Get,1.264372,0.625441,0.638931,47,0.032959,0.026902,0.013307,0.013594,0.071329,0.042218,0.219445
11,Uncontested Mark,1.341305,1.390682,-0.049377,124,0.086957,0.010817,0.011215,-0.000398,0.075669,0.093874,-0.016959
9,Shot,3.002739,2.927266,0.075473,17,0.011921,0.176632,0.172192,0.004440,0.169398,0.197596,0.025922
2,Free For,3.932048,2.097014,1.835034,28,0.019635,0.140430,0.074893,0.065537,0.221824,0.141552,0.630254


In [98]:
player = "Nick Daicos"
player_data = get_player(vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-1.948712,-1.133345,-0.815367,271,0.233018,-0.007191,-0.004182,-0.003009,-0.080914,-0.052942,-0.304623
7,Knock On,-0.110131,-0.108858,-0.001273,2,0.001720,-0.055066,-0.054429,-0.000636,-0.004573,-0.005085,-0.000476
1,Error,-0.097509,-0.082002,-0.015506,1,0.000860,-0.097509,-0.082002,-0.015506,-0.004049,-0.003831,-0.005793
10,Spoil,-0.007097,0.067831,-0.074928,3,0.002580,-0.002366,0.022610,-0.024976,-0.000295,0.003169,-0.027993
6,Kick,0.247126,0.356237,-0.109111,245,0.210662,0.001009,0.001454,-0.000445,0.010261,0.016641,-0.040764
5,Hard Ball Get,0.592866,0.612490,-0.019624,30,0.025795,0.019762,0.020416,-0.000654,0.024617,0.028611,-0.007332
3,Gather,1.497451,1.228978,0.268473,49,0.042132,0.030560,0.025081,0.005479,0.062176,0.057409,0.100302
11,Uncontested Mark,2.330194,2.461002,-0.130808,59,0.050731,0.039495,0.041712,-0.002217,0.096753,0.114961,-0.048870
2,Free For,2.420532,1.990375,0.430157,22,0.018917,0.110024,0.090472,0.019553,0.100504,0.092977,0.160708
8,Loose Ball Get,5.973515,4.099470,1.874045,144,0.123818,0.041483,0.028469,0.013014,0.248030,0.191499,0.700148


In [102]:
player = "Christian Petracca"
player_data = get_player(vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-1.724374,-0.323136,-1.401238,316,0.223164,-0.005457,-0.001023,-0.004434,-0.058886,-0.010934,5.204599
2,Error,-0.953795,-0.577560,-0.376235,6,0.004237,-0.158966,-0.096260,-0.062706,-0.032571,-0.019543,1.397443
11,Spoil,-0.078222,-0.172984,0.094762,5,0.003531,-0.015644,-0.034597,0.018952,-0.002671,-0.005853,-0.351974
1,Contested Mark,0.005635,0.019978,-0.014342,4,0.002825,0.001409,0.004994,-0.003586,0.000192,0.000676,0.053271
8,Knock On,0.048923,0.073120,-0.024197,6,0.004237,0.008154,0.012187,-0.004033,0.001671,0.002474,0.089874
10,Shot,2.461010,2.099611,0.361399,69,0.048729,0.035667,0.030429,0.005238,0.084041,0.071047,-1.342338
3,Free For,2.551978,2.315199,0.236780,16,0.011299,0.159499,0.144700,0.014799,0.087148,0.078342,-0.879467
0,Carry,3.061503,3.485388,-0.423885,232,0.163842,0.013196,0.015023,-0.001827,0.104548,0.117939,1.574431
7,Kick,3.535631,3.947957,-0.412326,288,0.203390,0.012276,0.013708,-0.001432,0.120739,0.133591,1.531496
4,Gather,3.630557,2.860236,0.770321,112,0.079096,0.032416,0.025538,0.006878,0.123980,0.096785,-2.861191


In [103]:
player = "Christian Petracca"
player_data = get_player(vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-2.077814,-1.202344,-0.875469,242,0.219005,-0.008586,-0.004968,-0.003618,-0.087432,-0.048232,0.752711
2,Error,-1.083767,-0.599471,-0.484296,11,0.009955,-0.098524,-0.054497,-0.044027,-0.045603,-0.024048,0.416389
11,Spoil,-0.102862,-0.119296,0.016435,5,0.004525,-0.020572,-0.023859,0.003287,-0.004328,-0.004786,-0.014130
8,Knock On,-0.053174,-0.063073,0.009899,2,0.001810,-0.026587,-0.031536,0.004949,-0.002237,-0.002530,-0.008511
1,Contested Mark,0.846780,0.859292,-0.012512,4,0.003620,0.211695,0.214823,-0.003128,0.035631,0.034471,0.010758
6,Hard Ball Get,1.463203,1.406198,0.057006,85,0.076923,0.017214,0.016544,0.000671,0.061570,0.056410,-0.049012
4,Gather,2.223042,2.018226,0.204816,76,0.068778,0.029251,0.026556,0.002695,0.093543,0.080962,-0.176097
0,Carry,2.323927,2.692867,-0.368940,167,0.151131,0.013916,0.016125,-0.002209,0.097788,0.108025,0.317207
3,Free For,2.393882,2.073823,0.320058,18,0.016290,0.132993,0.115212,0.017781,0.100731,0.083192,-0.275180
7,Kick,2.658495,3.422652,-0.764158,210,0.190045,0.012659,0.016298,-0.003639,0.111866,0.137301,0.657008


In [104]:
def get_team(vaep_data, team):
    
    return vaep_data[vaep_data['team'] == team]

In [106]:
team_data = get_team(vaep_2022, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-30.463250,-15.745593,-14.717656,3861,0.181310,-0.007890,-0.004078,-0.003812,-0.064484,-0.038029,-0.252120
2,Error,-7.961566,-4.615156,-3.346409,76,0.003569,-0.104757,-0.060726,-0.044032,-0.016853,-0.011147,-0.057326
12,Tackle,-0.058603,0.051686,-0.110289,3,0.000141,-0.019534,0.017229,-0.036763,-0.000124,0.000125,-0.001889
8,Knock On,9.400083,7.720155,1.679928,221,0.010378,0.042534,0.034933,0.007601,0.019898,0.018646,0.028778
4,Gather,15.891182,13.983864,1.907319,861,0.040432,0.018457,0.016241,0.002215,0.033638,0.033774,0.032673
11,Spoil,17.006615,-5.642432,22.649047,762,0.035783,0.022318,-0.007405,0.029723,0.035999,-0.013628,0.387989
7,Kick,19.244384,25.643691,-6.399307,4833,0.226955,0.003982,0.005306,-0.001324,0.040736,0.061935,-0.109623
6,Hard Ball Get,27.583160,15.987865,11.595295,1042,0.048932,0.026471,0.015343,0.011128,0.058387,0.038614,0.198633
1,Contested Mark,36.526462,28.526844,7.999619,387,0.018173,0.094384,0.073713,0.020671,0.077318,0.068899,0.137037
0,Carry,48.944175,48.612956,0.331219,3308,0.155342,0.014796,0.014696,0.000100,0.103604,0.117411,0.005674


In [107]:
team_data = get_team(vaep_2023, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-23.461594,-11.189236,-12.272358,2425,0.171197,-0.009675,-0.004614,-0.005061,-7.150204e-02,-0.038804,-0.308550
2,Error,-5.418049,-4.140301,-1.277747,45,0.003177,-0.120401,-0.092007,-0.028394,-1.651216e-02,-0.014359,-0.032125
12,Tackle,0.000324,0.065687,-0.065363,4,0.000282,0.000081,0.016422,-0.016341,9.883072e-07,0.000228,-0.001643
7,Kick,4.496164,14.743581,-10.247417,3322,0.234522,0.001353,0.004438,-0.003085,1.370260e-02,0.051131,-0.257639
8,Knock On,5.341748,4.574664,0.767083,106,0.007483,0.050394,0.043157,0.007237,1.627962e-02,0.015865,0.019286
4,Gather,12.283436,11.916276,0.367159,520,0.036710,0.023622,0.022916,0.000706,3.743525e-02,0.041326,0.009231
11,Spoil,18.128923,-0.731745,18.860669,532,0.037557,0.034077,-0.001375,0.035452,5.525008e-02,-0.002538,0.474192
6,Hard Ball Get,21.452597,14.338558,7.114039,682,0.048147,0.031455,0.021024,0.010431,6.537938e-02,0.049726,0.178860
0,Carry,28.388836,25.753625,2.635212,2112,0.149100,0.013442,0.012194,0.001248,8.651841e-02,0.089314,0.066254
1,Contested Mark,30.501323,26.328076,4.173246,294,0.020755,0.103746,0.089551,0.014195,9.295647e-02,0.091306,0.104923


In [108]:
team_data = get_team(vaep_2023, "Collingwood")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-23.272161,-13.120985,-10.151176,2561,0.184112,-0.009087,-0.005123,-0.003964,-0.072205,-0.046531,-0.251756
7,Kick,-5.594870,4.981442,-10.576312,3129,0.224946,-0.001788,0.001592,-0.003380,-0.017359,0.017666,-0.262299
2,Error,-5.372218,-3.678330,-1.693888,51,0.003666,-0.105338,-0.072124,-0.033213,-0.016668,-0.013045,-0.042010
12,Tackle,-0.017908,0.052008,-0.069917,3,0.000216,-0.005969,0.017336,-0.023306,-0.000056,0.000184,-0.001734
8,Knock On,5.855478,3.852961,2.002518,201,0.014450,0.029132,0.019169,0.009963,0.018168,0.013664,0.049664
4,Gather,12.941230,12.065718,0.875512,572,0.041121,0.022625,0.021094,0.001531,0.040152,0.042789,0.021713
11,Spoil,16.386329,-1.244890,17.631219,506,0.036377,0.032384,-0.002460,0.034844,0.050841,-0.004415,0.437266
6,Hard Ball Get,17.800628,11.105204,6.695424,574,0.041265,0.031012,0.019347,0.011665,0.055229,0.039383,0.166051
1,Contested Mark,28.091838,21.127825,6.964013,271,0.019482,0.103660,0.077962,0.025697,0.087159,0.074926,0.172712
0,Carry,43.541458,39.688795,3.852663,2300,0.165349,0.018931,0.017256,0.001675,0.135094,0.140749,0.095549


In [109]:
team_data = get_team(vaep_2023, "St Kilda")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
7,Kick,-21.106567,-9.061842,-12.044725,3328,0.235727,-0.006342,-0.002723,-0.003619,-0.077543,-0.040155,-0.258922
5,Handball,-20.838641,-11.769551,-9.069090,2555,0.180975,-0.008156,-0.004606,-0.003550,-0.076559,-0.052153,-0.194956
2,Error,-4.933897,-3.594992,-1.338905,54,0.003825,-0.091368,-0.066574,-0.024795,-0.018127,-0.015930,-0.028782
12,Tackle,0.592749,0.340167,0.252581,4,0.000283,0.148187,0.085042,0.063145,0.002178,0.001507,0.005430
8,Knock On,7.096831,5.853046,1.243785,150,0.010625,0.047312,0.039020,0.008292,0.026073,0.025936,0.026737
4,Gather,10.216783,9.397587,0.819196,534,0.037824,0.019133,0.017598,0.001534,0.037535,0.041643,0.017610
11,Spoil,14.263116,-2.360814,16.623929,499,0.035345,0.028583,-0.004731,0.033314,0.052401,-0.010461,0.357360
6,Hard Ball Get,20.274875,10.358877,9.915998,677,0.047953,0.029948,0.015301,0.014647,0.074488,0.045902,0.213162
1,Contested Mark,24.030672,18.406664,5.624008,212,0.015016,0.113352,0.086824,0.026528,0.088286,0.081564,0.120898
10,Shot,36.996907,36.889346,0.107561,365,0.025854,0.101361,0.101067,0.000295,0.135923,0.163465,0.002312
